<a href="https://colab.research.google.com/github/Siftee150/How_Many_topics/blob/master/Nematode_Biology/Research_Papers/paper-1/hungarian_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [502]:
import numpy as np
import random

In [503]:

def rcs(inter):
  flag=1 
  """
  flag acts as an indicator of whether row and column scanning operations are continuing to produce modified outputs on the given matrix
  If flag=0 across the whole loop,we will have to break out of loop 
  """
  while((inter==0).any()==True and flag==1):
    flag=0
    print("Row scanning")
    for row in inter:
      if(((row==0).sum())==1): #If only one zero is present across that row then mask the column correspondng to it and continue
        flag=1
        col_idex=np.where(row == 0)[0][0]
        m = np.zeros_like(inter)
        m[:,col_idex]=1
        inter=np.ma.MaskedArray(inter,m) #https://stackoverflow.com/questions/14611250/mask-specific-columns-of-a-numpy-array
    print(inter) #Result of row scanning
    
    print("Column scanning") 

    for j in range(len(inter)): #only one zero is present across that column,mask the row corresponding to it
      col=inter[:,j]
      if(((col==0).sum())==1):
        flag=1
        row_idex=np.where(col==0)[0][0]
        m=np.zeros_like(inter)
        m[row_idex,:]=1
        inter=np.ma.MaskedArray(inter,m)
    print(inter)    #Result of column scanning

  if (inter.mask.all()): #If all rows and columns are masked the lines are 3 and not 6
    return inter,3
  else:
    lines = np.count_nonzero(np.ma.count_masked(inter, axis=1)==len(inter))
    lines=lines+np.count_nonzero(np.ma.count_masked(inter, axis=0)==len(inter))
    return inter,lines

In [504]:
def o_a(inter,row_idex): 
  """
   The assignment will be where the 0's 
   are in the matrix such that only one 0 per row and column is part of the assignment.
  """ 

  # A recursive approach to find the optimum assignment of distinct positions for the matrix  
  
  if(row_idex>=len(inter)):
    a=[]
    return a
    
  for ele in np.where(inter[row_idex,:]==0)[0]:
    m = np.zeros_like(inter)
    m[:,ele]=1
    am=np.ma.MaskedArray(inter,m)
    a=o_a(am,row_idex+1)
    if (a != None):
      idx=str(row_idex)+","+str(ele)
      a.append(idx)
      return a
  return None







In [505]:
def check_random(inter):
  i,j=random.choice(np.argwhere(inter==0))
  m = np.zeros_like(inter)
  m[:,j]=1
  inter=np.ma.MaskedArray(inter,m)
  return inter

In [506]:
def hungarian_method(costmatrix,typ):
  inter=costmatrix
  if(typ=="Mx"):
    #converting the minimum cost bipartite problem to maximum cost bipartite problem
    print("converting the minimum cost bipartite problem to maximum cost bipartite problem")
    inter=-costmatrix+np.amax(costmatrix)
    print(inter)
  #else no change to inter array
  
  #Subtract the smallest value in each row from the other values in the row:
  print("Subtracting the minimum element along each row(axis=1) from all other elements in that row")
  inter=inter-np.amin(inter,axis=1)[:,None] #https://stackoverflow.com/questions/55251002/subtracting-minimum-of-row-from-the-row
  print(inter)
  
  #subtract the smallest value in each column from all other values in the column
  print("Subtracting the minimum element along each column(axis=0)  from all other elements in that column")
  inter=inter-np.amin(inter,axis=0)
  print(inter)
  
  spare=inter #storing the reduced array in spare matrix for future use before inter gets further changed

#Draw lines through the row and columns that have the 0 entries such that the fewest possible lines are drawn
  inter,lines=rcs(inter)
  if((inter==0).any()==True):
    inter=check_random(inter)
    inter,lines=rcs(inter)

  print("Min number of lines required to cover all zeros of array",str(lines))


  """If there are len(costmatrix) lines drawn, an optimal assignment of zeros is possible and the algorithm is finished. 
  If the number of lines is less than len(costmatrix), then the optimal number of zeroes is not yet reached. 
  To the next step.
  """
  while (lines<len(costmatrix)): #THE NEXT STEP
    """
    Find the smallest entry not covered by any line. 
    Subtract this entry from each row that isn’t crossed out, and then add it to each column that is crossed out.
    Then, go back to Step 3( def rcs() is STEP 3 ). 
    """
    min_t=np.amin(inter) #Find the smallest entry not covered by any line. 
    inter=inter-min_t  #subtracted from all non-masked elements i.e all elements not part of any crossed out row
    
    colns=np.argwhere(np.ma.count_masked(inter, axis=0)==len(inter))#count all crossed out columns
    print(colns) #display those columns(could strike this off as per your needs)
    rns=np.argwhere(np.ma.count_masked(inter, axis=1)==len(inter)) #count all crossed out rows
    print(rns)   #display those rows(could strike this off as per your needs)
    
    for ele in rns:
      for i in ele:
        for elem in colns:
          for j in elem:
            inter[i][j]=spare[i][j]+min_t #all the intersections of crossed out rows and columns will have the minimum element added to their original value
    
    inter.mask = np.ma.nomask  #remove all masked elements now

    print(inter)  #display the final matrix(could strike this off as per your needs)
    inter,lines=rcs(inter) #Then, go back to Step 3. 
    if((inter==0).any()==True):
      inter=check_random(inter)
      inter,lines=rcs(inter)

    print("Min number of lines to cover all zeros of array:",str(lines))
  inter.mask = np.ma.nomask
  print(inter)
    #finding assignment such that 0's are in the matrix such that only one 0 per row and column is part of the assignment
  oo_a=o_a(inter,0)
  print(oo_a)
  return oo_a 

    



In [507]:
"""
START OF THE CODE
"""
M=[[45,38,30,22],[35,29,20,14],[35,29,20,14],[27,20,15,10]]  #AN EXAMPLE ARRAY. MODIFIED AS PER YOUR CONVIENIENCE#
M=np.array(M) #convert to numpy array

typ=input("Maximisation or Minimisation problem?(Mx=>max Mn=>min) ")
f_a=hungarian_method(M,typ)
print("So final optimal assignment consists of:")
sum=0
for ele in f_a:
  a,b=ele.split(",")
  a=int(a)
  b=int(b)
  print(M[a][b],end=" ")
  sum=sum+M[a][b]
print("And final minimum total cost is:",str(sum))

Maximisation or Minimisation problem?(Mx=>max Mn=>minMx
converting the minimum cost bipartite problem to maximum cost bipartite problem
[[ 0  7 15 23]
 [10 16 25 31]
 [10 16 25 31]
 [18 25 30 35]]
Subtracting the minimum element along each row(axis=1) from all other elements in that row
[[ 0  7 15 23]
 [ 0  6 15 21]
 [ 0  6 15 21]
 [ 0  7 12 17]]
Subtracting the minimum element along each column(axis=0)  from all other elements in that column
[[0 1 3 6]
 [0 0 3 4]
 [0 0 3 4]
 [0 1 0 0]]
Row scanning
[[-- 1 3 6]
 [-- 0 3 4]
 [-- 0 3 4]
 [-- 1 0 0]]
Column scanning
[[-- 1 3 6]
 [-- 0 3 4]
 [-- 0 3 4]
 [-- -- -- --]]
Row scanning
[[-- -- 3 6]
 [-- -- 3 4]
 [-- -- 3 4]
 [-- -- -- --]]
Column scanning
[[-- -- 3 6]
 [-- -- 3 4]
 [-- -- 3 4]
 [-- -- -- --]]
Min number of lines required to cover all zeros of array 3
[[0]
 [1]]
[[3]]
[[0 1 0 3]
 [0 0 0 1]
 [0 0 0 1]
 [3 4 0 0]]
Row scanning
[[0 1 0 3]
 [0 0 0 1]
 [0 0 0 1]
 [3 4 0 0]]
Column scanning
[[0 1 0 3]
 [0 0 0 1]
 [0 0 0 1]
 [-- -- -- 